In [1]:
import nltk
from nltk.stem.porter import *
from nltk.classify import MaxentClassifier
import pickle
import os,sys
from io import open22
import string
from nltk.corpus import stopwords

ImportError: cannot import name 'open22'

In [ ]:
stop_words=set(stopwords.words('english'))
stopwordslist=[]
for stop_word in stop_words:
    stopwordslist.append(stop_word)

In [ ]:
def loadDataset(train_filename,testFilename):
    training_file = open(train_filename, "rt")
    testing_file = open(testFilename, "rt")
    return training_file, testing_file

In [ ]:
def generateTagDict(train):
    i=1
    tagDict={}
    change_of_sentence_flag=0
    for line in train:
        s = re.match(r'^\s*$', line)  #find empty line
        if(s):
            change_of_sentence_flag = 1
        else: 
            sentenceList = line.split()
            tag =  sentenceList[1]
            #store words that are begining of the sentence
            if(change_of_sentence_flag == 1):
                change_of_sentence_flag = 0
            if(tag not in  tagDict):
                tagDict[tag]= i
                i=i+1
    return tagDict


def MEMM_features(word ,previous_tag, tagIntegerMap):
    stemmer = PorterStemmer() 
    features = []
    #checking for upper case
    if(word[0].isupper()):
        features.append(1)
    else:
        features.append(0)
    #Checking whether word is start word 
    if(word in wordStartList):
        features.append(1)
    else:
        features.append(0)
        
    #Checking for word stating with capital character and not a start word
    if(word not in wordStartList and word[0].isupper()):
        features.append(1)
    else:
        features.append(0)
     
    try:
        if(float(word)):
            features.append(1)
    except:
        features.append(0)
    #features['term_frequency'] = my_dict[word]
    
    #Appending previous_tag in feature set
    if(previous_tag in tagIntegerMap):
        features.append(tagIntegerMap[previous_tag])
    else:
         features.append(0)
            
    #check if word ends with ing
    if(word.endswith('ing')):
        features.append(1)
    else:
        features.append(0)
    #check if word ends with ed
    if(word.endswith('ed')):
        features.append(1)
    else:
        features.append(0)
    #check is word ends wih ly
    if(word.endswith('ly')):
        features.append(1)
    else:
        features.append(0)
    
    #word length
    features.append(len(word))
    #check if word is a stop word:
    if(word in stopwordslist):
        features.append(1)
    else:
        features.append(0)
        
    #check if word is a punctuation:
    if(word in string.punctuation):
        features.append(1)
    else:
        features.append(0)

    return features

In [ ]:
change_of_sentence_flag = 0 #a marker for the end of sentence
wordStartList = [] #store words that are begining of the sentence
training,testing=loadDataset("train.txt","test.txt")
tagDict = generateTagDict(training)
training = open("train.txt", "rt")
train_features = generateMatrix(training)
len(train_features)

In [ ]:
train_labeled_featuresets = [(MEMM_features(word,previous_tag,tagDict),tagDict[tag]) for (word,previous_tag,tag) in train_features]
trainSet=[]
y_train = []
for i in range(len(train_labeled_featuresets)):
    trainSet.append(train_labeled_featuresets[i][0])
    y_train.append(train_labeled_featuresets[i][1])

In [ ]:
test_features = generateMatrix(testing)
test_labeled_featuresets = [(MEMM_features(word,previous_tag,tagDict),tagDict[tag]) for (word,previous_tag,tag) in test_features]
testSet=[]
y_test = []
for i in range(len(test_labeled_featuresets)):
    testSet.append(test_labeled_featuresets[i][0])
    y_test.append(test_labeled_featuresets[i][1])

In [ ]:
from sklearn.linear_model import LogisticRegression
import math

classifier = LogisticRegression(multi_class='multinomial', solver='saga')
classifier.fit(trainSet,y_train)

In [ ]:
def MEMM(wordList):
    viterbi = [[0 for x in range(len(wordList))] for x in range(len(tagDict))] 
    backpointer = [['' for x in range(len(wordList))] for x in range(len(tagDict))] 
    '''startWord=wordList[0]
    featureVector = MEMM_features(startWord,0,tagDict)
    #print(type(featureVector))
    startProbablity=classifier.predict_proba([featureVector])
    for i in range(len(tagDict)):
        viterbi[i][0]= startProbablity[0][i]'''
        
    for wordIndex in range(len(wordList)):
        for classIndex in range(len(classifier.classes_)):
            if(wordIndex==0):
                featureVector = MEMM_features(wordList[wordIndex],0,tagDict)
            else:
                featureVector = MEMM_features(wordList[wordIndex],classifier.classes_[classIndex],tagDict)
            probablity=classifier.predict_proba([featureVector])
            if(wordIndex!=0):
                for i in range(len(tagDict)):
                    if(viterbi[i][wordIndex] < probablity[0][i] * viterbi[classIndex][wordIndex-1]):
                        viterbi[i][wordIndex]= probablity[0][i] * viterbi[classIndex][wordIndex-1]
                        #index of tag at which maximum value is obtained
                        backpointer[i][wordIndex] = classIndex
            else:
                for i in range(len(tagDict)):
                    viterbi[i][0]= probablity[0][i]
                    backpointer[i][0]=0
    
    maxValue = 0
    backIndex=0
    for index in range(len(classifier.classes_)):
        if(maxValue < viterbi[index][len(wordList)-1]):
            maxValue = viterbi[index][len(wordList)-1]
            backIndex = index
   
    predictedTagList=[]
    predictedTagList.append(backIndex+1)
    for wordIndex in range(len(wordList)-1,0,-1):
        backIndex = backpointer[backIndex][wordIndex]
        predictedTagList.append(backIndex+1)
   
    # returning the predicted tags for a particular sentence
    return predictedTagList[::-1]
    
    

    #for j in range(len(wordList)):

In [ ]:
wordList = [] #store words in a sentence
tagList = [] #store part-of-speech tag in a sentence 
#prob_table = {} #stpre the posterior
y_pred=[]
input_file = open('test.txt', "rt")
for line in input_file:
    if line.strip() != '': #if not empty do following 
        sentenceList = line.split()
        word = sentenceList[0]
        tag = sentenceList[1]
        wordList.append(word)
        tagList.append(tag)
        if change_of_sentence_flag == 1:
            wordStartList.append(word)
            change_of_sentence_flag = 0
    s = re.match(r'^\s*$', line)  #find empty line
    if s:
        #print (wordList)
        change_of_sentence_flag = 1
        predictedTagList = MEMM(wordList) #list of tagsreturned by HMM function call
        #for i in range(len(wordList)): #part_of_speech_tag(tagList) and token_list(wordList) has the same length
         #   print(wordList[i]+" "+ tagList[i]+ " " + " " + path[i] + "\n")
        y_pred= y_pred + predictedTagList
        wordList = [] # refresh word list
        tagList = []
        boiList = []
        #prob_table = {}#refresh prob_table

In [ ]:
from sklearn.metrics import accuracy_score
print('accuracy is : ',accuracy_score(y_test,y_pred)*100)

In [ ]:
import numpy as np
predictedTagList=MEMM(wordList)